In [78]:
# imports

import os
from dotenv import load_dotenv
from IPython.display import Markdown, display
from openai import OpenAI

In [ ]:
!pip install youtube_transcript_api

In [79]:
from youtube_transcript_api import YouTubeTranscriptApi

In [80]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

In [92]:
import re

class YouTubeWebLink:
    def __init__(self, url):
        self.url = url
        self.video_id = self.get_video_id(url)
        self.set_openai_client()
        self.set_system_prompt()

    def get_video_id(self, url):
        """ extract youtube video id from url with regular expression """
        regex = r"(?:v=|be/)([a-zA-Z0-9_-]{11})"
        match = re.search(regex, url)
        if match:
            return match.group(1)
        else:
            raise ValueError("Probably not a YouTube URL")
        
    def set_openai_client(self):
        self.openai = OpenAI()
        
    def set_system_prompt(self, system_prompt=None):
        """ set system prompt from youtube video """
        self.system_prompt = """
        You are a skilled explainer and storyteller who specializes in summarizing YouTube video transcripts in a way that's both engaging and informative. 
        Your task is to:
        - Capture key points and main ideas of the video
        - Structure your summary with in clear sections
        - Include important details, facts, and figures mentioned
        - Never end your summary with a "Conclusion" section
        - Keep the summary short and easy to understand
        - Always format your response in markdown for better readability
        """ if system_prompt is None else system_prompt

    def get_transcript(self):
        """ get transcript from youtube video """
        try:
            print('Fetching video transcript...')
            transcript = YouTubeTranscriptApi.get_transcript(self.video_id)
            return " ".join([item['text'] for item in transcript])
        except Exception as e:
            print(f"Error fetching transcript: {e}")
            return None
        
    def get_summary_from_transcript(self, transcript):
        """ summarize text using openai """
        try:
            print('Summarizing video...')
            response = self.openai.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": self.system_prompt},
                    {"role": "user", "content": f"Summarize the following YouTube video transcript:\n\n{transcript}"}
                ]
            )
            return response.choices[0].message.content
        except Exception as e:
            print(f"Error summarizing text: {e}")
            return None

    def display_summary(self):
        """ summarize youtube video """
        transcript = self.get_transcript()
        summary = self.get_summary_from_transcript(transcript)
        display(Markdown(summary))


In [93]:
# video link and share link of same youtube video
test_url_1 = "https://www.youtube.com/watch?v=nYy-umCNKPQ&list=PLWHe-9GP9SMMdl6SLaovUQF2abiLGbMjs"
test_url_2 = "https://youtu.be/nYy-umCNKPQ?si=ILVrQlKT0W71G5pU"


In [ ]:
# Test that we get same id
video1, video2 = YouTubeWebLink(test_url_1), YouTubeWebLink(test_url_2)
video1.video_id, video2.video_id

In [ ]:
video1.display_summary()